# Q3 Currency Formatter
TASK: Currency Formatter & Rounding Ties-to-Even. Write fmt_money(x: float) -> str that returns a string like 1,234.50 using proper rounding (banker’s rounding / ties-to-even). Include test cases that demonstrate ties (e.g., 2.5, 3.5). Explain 2 floating-point representation error and how formatting mitigates it for display. Anchor concepts: numbers, round, representation error, formatted printing. Reference: :contentReference[oaicite:8]index=8


# PROMPT
Write fmt_money(x: float) -> str that formats with thousands separators and two decimals using banker’s rounding (ties-to-even). Avoid float rounding pitfalls. Include tests that show ties (e.g., 2.345 vs 2.355) and the classic 2.675 case.

In [1]:
#CHATGPT CODE
def fmt_money(x):
    
    return f"{round(x, 2):,.2f}"

print(fmt_money(1234.5))
print(fmt_money(2.675))



1,234.50
2.67


# CRITIQUE 
Correctness: Python’s round is half‑even, but binary float representation can give surprising results (e.g., round(2.675, 2) → 2.67).

Complexity: O(1).

Robustness: No handling for non-numeric inputs.

Readability/Style: Add type hints, docstring, and tests.

Lecture faithfulness: Should address representation error explicitly (use Decimal(str(x))) and format



# FLOATING POINT NUMBERS

Floating‑point numbers in Python use IEEE‑754 binary, so many simple decimal fractions (like 0.1 or 2.675) cannot be represented exactly. Those tiny representation gaps can accumulate or surface when you add, compare, or round, producing results like 0.1 + 0.2 == 0.30000000000000004. Python’s round implements ties‑to‑even, but because the stored value may be slightly below the intuitive decimal, a value that looks halfway can round down (e.g., round(2.675, 2) → 2.67). For user‑facing output, formatted printing hides harmless noise by constraining what you display, such as using f"{x:.2f}" or f"{x:,.2f}". When you require decimal‑exact behavior—especially in finance—construct Decimal from strings and quantize to the needed precision with a clear rounding rule like ROUND_HALF_EVEN. This combination—understanding binary representation, using round appropriately, and formatting or Decimal for display—meets the midterm’s emphasis on numbers, representation error, rounding, and formatted printing.


In [2]:
#IMPROVED CODE
from decimal import Decimal, ROUND_HALF_EVEN, InvalidOperation
from typing import Union

Number = Union[float, int, str, Decimal]


def fmt_money(x: Number) -> str:
    """
    Format a number with thousands separators and two decimal places,
    using banker's rounding (ties-to-even).

    - Accepts float, int, Decimal, or numeric string.
    - Converts input to Decimal using its string representation to avoid
      float rounding artifacts.
    - Returns a formatted string like '1,234.56'.
    """
    # Safe Decimal conversion
    if isinstance(x, Decimal):
        d = x
    else:
        try:
            d = Decimal(str(x))
        except (InvalidOperation, ValueError, TypeError):
            raise TypeError("Input must be a number or numeric string.")

    # Banker's rounding to two decimals
    rounded = d.quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)
    return f"{rounded:,.2f}"


def main() -> None:
    """Print formatted currency strings for four example inputs.
       Prints four formatted examples using ties-to-even rounding.
    """
    # Silent tests (assertions)
    assert fmt_money(1234.5) == "1,234.50"
    assert fmt_money(2.345) == "2.34"
    assert fmt_money(2.355) == "2.36"
    assert fmt_money(2.675) == "2.68"

    print(fmt_money(1234.5))
    print(fmt_money(2.345))
    print(fmt_money(2.355))
    print(fmt_money(2.675))


if __name__ == "__main__":
    main()

    # --- Additional demonstration tests ---
    print("\nAdditional Tests:")
    examples = [
        (150000, "150,000.00"),       # thousands separator
        (1234567.895, "1,234,567.90"),
        ("2.675", "2.68"),            # string input
        (Decimal("2.675"), "2.68"),   # Decimal input
        (0.005, "0.00"),              # ties-to-even (5 -> even)
        (0.015, "0.02"),              # ties-to-even (1 -> odd)
    ]

    for val, expected in examples:
        out = fmt_money(val)
        print(f"  Input: {val!r:12} -> Output: {out:12}  (Expected: {expected})")
        assert out.endswith(expected), f"Expected {expected}, got {out}"

    print("\nAll tests passed successfully")


1,234.50
2.34
2.36
2.68

Additional Tests:
  Input: 150000       -> Output: 150,000.00    (Expected: 150,000.00)
  Input: 1234567.895  -> Output: 1,234,567.90  (Expected: 1,234,567.90)
  Input: '2.675'      -> Output: 2.68          (Expected: 2.68)
  Input: Decimal('2.675') -> Output: 2.68          (Expected: 2.68)
  Input: 0.005        -> Output: 0.00          (Expected: 0.00)
  Input: 0.015        -> Output: 0.02          (Expected: 0.02)

All tests passed successfully
